In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
!pip install -r requirements.txt


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [3]:
from src.data_loader import (
    load_lyrics,
    load_user_top_tracks,
    load_user_top_artists,
    load_user_top_albums,
)

df = load_lyrics()
df_tracks = load_user_top_tracks()
df_artists = load_user_top_artists()
df_albums = load_user_top_albums()

df.head()

df_tracks.head()

,user_id,rank,track_name,artist_name,playcount,mbid
0,1,1,Ice Cream,New Young Pony Club,54,417a3d0b-0291-4e08-b167-50971dbdae36
1,1,2,Hiding On The Staircase,New Young Pony Club,51,f6ac2fd2-b33b-48ff-a094-537cf530cf20
2,1,3,Descend,New Young Pony Club,49,33434fd1-79a1-3882-bb31-fbd430815a93
3,1,4,Air War,Crystal Castles,48,06d3fea3-e806-35ed-9562-aa396a97de14
4,1,5,Get Lucky,New Young Pony Club,48,ab351b25-7126-3792-9b0d-ce44fd0c0462


In [4]:
print(df_tracks.loc[df_tracks["track_name"] == "You Reposted in the Wrong Neighborhood"])

          user_id  rank                              track_name artist_name  \
22254538   447927    39  You Reposted in the Wrong Neighborhood      ＳＨＯＫＫ青   

          playcount mbid  
22254538         68  NaN  


In [5]:
if("link" in df):
    df = df.drop("link", axis = 1).reset_index(drop = True)
df = df.sample(5000).reset_index(drop=True)

import sys
import os

# Ensure project root is on sys.path
sys.path.append(os.path.abspath("."))

from src.text_processor import TextProcessor
from src.recommender import ContentRecommender, CollaborativeRecommender

processor = TextProcessor()
df = processor.normalize_text(df, "text")


In [6]:
df['text'].iloc[1]

"oh danny boy, the pipes, the pipes are calling  \r from glen to glen and down the mountain side  \r the summer's gone and all the roses falling  \r 'tis you, 'tis you must go and i must bye  \r   \r oh danny boy, oh danny boy the pipes are calling  \r from glen to glen and even down the mountain side  \r the summer's gone and all the roses falling  \r 'tis you, 'tis you must go and i must bye  \r   \r but come ye back when summer's in the meadow  \r or when the valleys hushed and white with snow  \r and i'll be here in sunshine or in shadow  \r oh danny boy, oh danny boy i love you so  \r   \r oh danny boy, oh danny boy the pipes are calling  \r from glen to glen and even down the mountain side  \r the summer's gone and all the roses falling  \r 'tis you, 'tis you must go and i must bye  \r   \r but come ye back when summer's in the meadow  \r or when the valleys hushed and white with snow  \r and i'll be here in sunshine or in shadow  \r oh danny boy, oh danny boy i love you so  \r o

In [7]:
# Tokenization example
print(processor.tokenize("my beautiful, beauty"))

my beauti , beauti


In [8]:
from sklearn.metrics.pairwise import cosine_similarity

matrix = processor.get_tfidf_matrix(df["text"])
similar = cosine_similarity(matrix)
similar[0]

array([1.        , 0.00109121, 0.00492448, ..., 0.05267451, 0.03379421,
       0.00746998])

In [9]:
print(df.loc[df["artist"] == "ABBA"])

     artist                              song  \
26     ABBA             Fernando (In Spanish)   
943    ABBA                  Bang-A-Boomerang   
1770   ABBA       Slipping Through My Fingers   
1949   ABBA                 Tropical Loveland   
2695   ABBA           Sitting In The Palmtree   
2769   ABBA             Pick A Bale Of Cotton   
2782   ABBA                     Hey Hey Helen   
2855   ABBA                 Rock 'n Roll Band   
3929   ABBA                   Love Isn't Easy   
3965   ABBA                          Soldiers   
4029   ABBA                      The Visitors   
4074   ABBA  Me And Bobby And Bobby's Brother   
4105   ABBA                         That's Me   
4432   ABBA          Givin' A Little Bit More   
4801   ABBA                 Summer Night City   
4806   ABBA                      My Mama Said   

                                                   text  
26    puedes escuchar fernando?  \r me recuerda tiem...  
943   making somebody happy is a question of give 

In [10]:
print(df.loc[df["artist"] == "ABBA"])

     artist                              song  \
26     ABBA             Fernando (In Spanish)   
943    ABBA                  Bang-A-Boomerang   
1770   ABBA       Slipping Through My Fingers   
1949   ABBA                 Tropical Loveland   
2695   ABBA           Sitting In The Palmtree   
2769   ABBA             Pick A Bale Of Cotton   
2782   ABBA                     Hey Hey Helen   
2855   ABBA                 Rock 'n Roll Band   
3929   ABBA                   Love Isn't Easy   
3965   ABBA                          Soldiers   
4029   ABBA                      The Visitors   
4074   ABBA  Me And Bobby And Bobby's Brother   
4105   ABBA                         That's Me   
4432   ABBA          Givin' A Little Bit More   
4801   ABBA                 Summer Night City   
4806   ABBA                      My Mama Said   

                                                   text  
26    puedes escuchar fernando?  \r me recuerda tiem...  
943   making somebody happy is a question of give 

In [11]:
collab_recommender = CollaborativeRecommender(df_tracks, df_artists, df_albums)

# Example: User Similarity
print("Similarity between User 5 and 7:", collab_recommender.calculate_user_similarity(5, 7))

# Top Songs for User 5
print("\nTop Songs for User 5:", collab_recommender.get_top_items(5, 'songs'))

Similarity between User 5 and 7: 11

Top Songs for User 5: ['Go Long', 'Skinny Love', 'Emily', 'Kid A', 'Lump Sum', 'Only Skin', 'Sawdust & Diamonds', 'Sleeping Lessons', '2080', 'This Time Tomorrow', 'My Girls', 'Flume', 'These Days', 'Monkey & Bear', 'Helicopter', 'Paris', 'The Next Time Around', "You Don't Know Her Name", 'Viðrar vel til loftárása', 'Between the Bars', 'Woods', 'Jacksonville', 'Debaser', 'Neighborhood #1 (Tunnels)', 'Osaka Loop Line', 'Time to Pretend', 'The Love Quadrant', "She's A Rainbow", 'Mansard Roof', 'Chicago', 're: stacks', 'White Winter Hymnal', 'Cape Cod Kwassa Kwassa', 'Dull Life', 'Triangle Walks', "(I Can't Get No) Satisfaction", 'Monkey Gone to Heaven', 'Bridges and Balloons', 'Walking on a Dream', 'Sleepyhead', "Let's Dance", 'Sunday Morning', 'Acolyte', 'Here Comes the Sun', 'Heads Will Roll', 'Ruby Tuesday', "I'm New Here", 'Stephanie Says', "'81", 'Harder, Better, Faster, Stronger']


In [12]:
# Calculate similarity using the refactored TF-IDF method
user_id_1 = 10
user_id_2 = 10

# Self-similarity check
print(f"Self-similarity for User {user_id_1}: {collab_recommender.calculate_tfidf_user_similarity(user_id_1, user_id_2)}")

# Find similar users based on TF-IDF ratio threshold
target_user = 87
threshold = 0.12

similar_users = collab_recommender.get_similar_users_tfidf(target_user, threshold=threshold)
print(f"Users similar to User {target_user} (threshold {threshold}): {similar_users}")

Self-similarity for User 10: 183.0192237702804
Users similar to User 87 (threshold 0.12): [1, 4, 5, 7, 12, 14, 15, 18, 19, 20, 21, 22, 24, 26, 27, 28, 31, 32, 33, 35, 41, 42, 43, 44, 46, 49, 50, 51, 53, 54, 55, 58, 59, 62, 63, 66, 67, 68, 69, 71, 72, 73, 74, 75, 77, 78, 80, 81, 83, 89, 91, 92, 93, 94, 96, 97, 99, 100]


In [13]:
# Recommend new music based on overlap similarity
# This replaces the old 'compare_users' function
target_user_id = 220

# Using the class method which implements similar logic (finding users with high overlap)
recommendations = collab_recommender.recommend_new_music(target_user_id, threshold=10)

print(f"Recommended Songs for User {target_user_id}:")
print(recommendations['songs'][:20])  # Print top 20 for brevity

Recommended Songs for User 220:
["Message in a Bottle (Taylor's Version) (From the Vault)", 'Potential Breakup Song', 'Comme des Garçons (Like the Boys)', 'Welcome to New York', "All Too Well (10 Minute Version) (Taylor's Version) (From the Vault)", 'Live to Tell', 'BREAK MY SOUL', 'Ribs', 'West Coast', 'Cruel Summer', 'Alibi (with Pabllo Vittar & Yseult)', 'Triste com T', 'Judas', 'Miss Americana & the Heartbreak Prince', 'XS', '9 Crimes', "You're on Your Own, Kid", 'Wildest Dreams', 'Frozen', 'Delicate']


In [14]:
# Check shared items between specific user pairs
pairs = [(5, 21), (5, 24), (21, 24)]

for u1, u2 in pairs:
    shared_songs = collab_recommender.get_shared_items(u1, u2, 'songs')
    shared_artists = collab_recommender.get_shared_items(u1, u2, 'artists')
    shared_albums = collab_recommender.get_shared_items(u1, u2, 'albums')
    
    print(f"\n{u1} and {u2}:")
    print(f"Songs: {shared_songs}")
    print(f"Artists: {shared_artists}")
    print(f"Albums: {shared_albums}")


5 and 21:
Songs: ['My Girls', 'Mansard Roof', 'Chicago', 'Cape Cod Kwassa Kwassa']
Artists: ['The Shins', 'Arcade Fire', 'Sufjan Stevens', 'Vampire Weekend', 'Belle and Sebastian', 'Sigur Rós', 'The Strokes', 'The Beatles', 'MGMT', 'Lady Gaga', 'She & Him', 'Animal Collective']
Albums: ['Funeral', 'Merriweather Post Pavilion', 'Oracular Spectacular', 'Neon Bible', 'Vampire Weekend', 'Back to Black']

5 and 24:
Songs: ['Paris']
Artists: ['Bon Iver', 'The Shins', 'Arcade Fire', 'Yeah Yeah Yeahs', 'Regina Spektor', 'Belle and Sebastian', 'Kings of Leon', 'The Strokes', 'The Beatles', 'M.I.A.', 'Beck', 'The Decemberists']
Albums: ['For Emma, Forever Ago', 'Fleet Foxes', 'Lungs', 'Neon Bible']

21 and 24:
Songs: []
Artists: ['Belle and Sebastian', 'Arcade Fire', 'The Beatles', 'The Strokes', 'Oasis', 'Arctic Monkeys', 'The Shins', 'Eels']
Albums: ['The Suburbs', 'Neon Bible', 'The Life Pursuit', 'The Bird Of Music']


In [15]:
def common_users(songs = [], artists = [], albums = []):
    temp = []
    for song in songs:
        print(len(df_tracks.loc[df_tracks["track_name"] == song, "user_id"].tolist()))
        temp.append(df_tracks.loc[df_tracks["track_name"] == song, "user_id"].tolist())
    for artist in artists:
        temp += (df_artists.loc[df_artists["artist_name"] == artist, "user_id"].tolist())
    for album in albums:
        temp += (df_albums.loc[df_albums["album_name"] == album, "user_id"].tolist())
    intersection_set = set.intersection(*[set(lst) for lst in temp])

    # Convert the resulting set back to a list if needed
    intersection_list = list(intersection_set)
    return intersection_list

test_song = "I'm Dancing in the Show Tonight"
test_song_2 = "Jingle Bells"

test_artist_1 = "Eminem"
test_artist_2 = "Toby Fox"

test_album_1 = "Fleet Foxes"
test_album_2 = "Wincing the Night Away"

print(common_users(songs = [test_song, test_song_2]))

## print("Next\n\n")

## print(common_users(artists = [test_artist_1, test_artist_2], albums = [test_album_1, test_album_2]))

136
207
[339087]


In [16]:
from src.recommender import CollaborativeRecommender
import random

def random_choices(songs, N):
    temp = []
    users = []

    # 1. Collect users who have these songs
    for song in songs:
        users += df_tracks.loc[df_tracks["track_name"] == song, "user_id"].tolist()

    if len(users) < 2:
        return []

    attempts = 0
    while len(temp) < N and attempts < 100:
        attempts += 1

        # 2. Pick 2 random users
        r1, r2 = random.sample(users, 2)

        # 3. Get shared songs via the recommender
        rand_songs = collab_recommender.get_shared_items(r1, r2, "songs")

        if len(rand_songs) == 0:
            continue

        # 4. Pick a random shared song
        rand_song = random.choice(rand_songs)
        if rand_song not in temp and rand_song not in songs:
            temp.append(rand_song)

    return temp

print(random_choices(["Fly Me To the Moon", "Mack the Knife", "Runaround Sue"], 5))

['The Story of My Life', 'The Sound of Silence', 'Stand by Me', 'Runaway', 'Born in the U.S.A.']
